# 9. 인물사진을 만들어보자

테스트할 이미지파일과 학습 모델 파일 경로 설정
```shell
# 노드 기준
$ mkdir -p ~/aiffel/human_segmentation/models
$ mkdir -p ~/aiffel/human_segmentation/images
$ ln -s ~/data/* ~/aiffel/human_segmentation/images

# 로컬 기준
## 이미지 파일(my_image.png) 위치
$ echo ~/data/human_segmentation/images/my_image.png
$ echo ~/data/human_segmentation/models/deeplabv3_xception_tf_dim_ordering_tf_kernels.h5
```

## 9-2. 셸로우 포커스 만들기
### (1) 사진을 준비하자

![how to make shallow focus with single camera](../images/13.png)</br>

두 개의 렌즈가 맡은 역할을 하나의 렌즈에서 구현해야함</br>
1. 이미지 세그멘테이션(image segmentation)
    - 하나의 이미지에서 배경과 사람을 분리
2. 분리된 배경을 흐리게(blur) 처리
3. 사람 이미지와 `2번` 결과로 생성된 분리된 흐린 배경 이미지를 합치기
    - 최종적으로 아웃포커싱 효과가 적용된 인물 사진을 획득


사진을 준비하고 python module을 추가하자

In [ ]:
# os: Operating System의 줄임말로, 운영체제에서 제공되는 여러 기능을 파이썬에서 사용할 수 있도록 함 (Ex. 디렉토리 경로 이동, 시스템 환경 변수 가져오기 등)
# urllib: URL 작업을 위한 여러 모듈을 모은 패키지. (Ex. urllib.request, urllib.parse, ...)
# cv2: OpenCV 라이브러리로, 실시간 컴퓨터 비전을 목적으로 한 프로그래밍 라이브러리
# numpy(NumPy): 행렬이나 대규모 다차원 배열을 쉽게 처리할 수 있도록 지원하는 라이브러리. 데이터 구조 외에도 수치 계산을 위해 효율적으로 구현된 기능을 제공
# pixellib: 이미지 및 비디오 segmentation을 수행하기 위한 라이브러리. 
# pixellib.semantic: segmentation 기법 중, semantic segmentation을 쉽게 사용할 수 있도록 만든 라이브러리
# matplotlib: 파이썬 프로그래밍 언어 및 수학적 확장 NumPy 라이브러리를 활용한 플로팅 라이브러리로, 데이터 시각화 도구
import os
import urllib
import cv2
import numpy as np
from pixellib.semantic import semantic_segmentation
from matplotlib import pyplot as plt

print('슝=3')

```shell
# 로컬 기준 버전 에러 등 여러 이슈가 있어 노드의 결과를 그대로 작성 또는 캡쳐함
Matplotlib is building the font cache; this may take a moment.


슝=3
```

In [ ]:
# os 모듈에 있는 getenv() 함수를 이용하여 읽고싶은 파일의 경로를 file_path에 저장
# 준비한 이미지 파일의 경로를 이용하여, 이미지 파일을 읽음
# cv.imread(경로): 경로에 해당하는 이미지 파일을 읽어서 변수에 저장
## 노드 기준 파일 패스
img_path = os.getenv('HOME')+'/aiffel/human_segmentation/images/my_image.png'
img_orig = cv2.imread(img_path)

print(img_orig.shape)

# cv2.cvtColor(입력 이미지, 생상 변환 코드): 입력 이미지의 색상 채널을 변경
# cv2.COLOR_BGR2RGB: 이미지 색상 채널을 변경 (BGR 형식을 RGB 형식으로 변경)
# plt.imshow(): 저장돈 데이터를 이미지의 형식으로 표시, 입력은 RGB(A) 데이터 혹은 2D 스칼라 데이터
# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html
# plt.show(): 현재 열려있는 모든 figure를 표시 (여기서 figure는 이미지, 그래프 등)
# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.show.html
plt.imshow(cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB))
plt.show()

```shell
(500, 892, 3)
```
![Read example image](../images/14.png)</br>

## 9-3. 셸로우 포커스 만들기
### (2) 세그멘테이션으로 사람 분리하기

배경에만 렌즈 흐림 효과를 주기 위해서 그림처럼 이미지에서 사람과 피사체를 분리해야한다.</br>
흔히 포토샵으로 `누끼 따기`라는 것이 이런 분리 작업이다.</br>
![image segmentation example](../images/15.png)</br>

### **세그멘테이션(Segmentation)?**
**이미지 세그멘테이션(image segmentation)**
- 이미지에서 픽셀 단위로 관심 객체를 추출하는 방법

시멘틱 세그멘테이션(semantic segmentation)
- 우리가 인식하는 세계처럼 물리적 의미 단위로 인식하는 세그멘테이션 기법
    - 이미지에서 픽셀을 사람, 자동차, 비행기 등의 물리적 단위로 분류(classification)하는 방법

인스턴스 세그멘테이션(instance segmentation)
- 시멘틱 세그멘테이션에서는 `사람`을 추출할 때 여러 객체(`사람`)가 있더라도 같은 라벨로 표현이 된다.
    - 인스턴스 세그멘테이션은 사람 개개인별로 다른 라벨을 가지게 한다.
    - 여러 사람이 한 이미지에 등장할 때 객체를 분할해서 인식하자는 것이 목표

![MS COCO 데이터셋 라벨 example](../images/16.png)</br>

## 딥러닝 이전 방식 - 이미지 세그멘테이션
### 워터쉐드 세그멘테이션(watershed segmentation)

이미지를 그레이스케일(grayscale)로 변환해서 픽셀 값의 높고 낮음을 표시하고 낮은 부분부터 채워나가면서 경계선(`각 영역에서 물이 차오르다가 넘치는 곳`)을 만들어 물체를 구분하는 방식</br>

참고링크: [opencv-python tutorial](https://opencv-python.readthedocs.io/en/latest/doc/27.imageWaterShed/imageWaterShed.html)

## 9-4. 셸로우 포커스 만들기
### (3) 시맨틱 세그멘테이션 다뤄보기

세그멘테이션 문제에는 FCN, SegNet, U-Net 등 많은 모델이 사용된다.</br>
그 중에서 `DeepLab` 모델을 사용하여 예제를 수행한다. (DeepLab v3+ 알고리즘은 세그멘테이션 모델 중에서도 성능이 매우 좋아 최근까지도 많이 사용되고 있다.)

DeepLab에 대한 상세설명: [DeepLab V3+: Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation](http://bloglunit.wordpress.com/2018/07/02/deeplab-v3-encoder-decoder-with-atrous-separable-convolution-for-semantic-image-segmentation)

노드 짜투리 문제
```text
Q. DeepLab에서 atrous convolution을 사용한 이유가 무엇인가?
A. receptive field를 넓게 사용하기 위해 (적은 파라미터로 필터가 더 넓은 영역을 보게 하기 위해)

Q. Depthwise separable convolution은 어디서 처음 사용한 것일까요? 왜 이구조가 효율적인가요?
A. Xception 에서 제안되었다. 3x3 conv layer의 receptive field를 1/9 수준의 파라미터로 구현할 수 있어 효율적이다.
```

<details>
<summary>참고자료</summary>

[Xception 더 알아보기](https://arxiv.org/abs/1610.02357)
</details>

DeepLab 모델을 준비. `PixelLib`를 이용하면 편하게 사용가능</br>
- [PixelLib](https://github.com/ayoolaolafenwa/PixelLib)

In [ ]:
# 저장할 파일 이름을 결정합니다.
# 1. os.getenv(x) 함수는 환경 변수 x의 값을 ㅗ함하는 문자열 변수를 반환합니다.
## model_dir에 "/aiffel/human_segmentation/models" 저장
model_dir = os.getenv('HOME')+'/aiffel/human_segmenation/models'
# 2. os.path.join(a, b)는 경로를 병합하여 새 경로 생성
## model_file에 "/aiffel/aiffel/human_segmentation/models/deeplabv3_xception_tf_dim_ordering_tf_kernels.h5" 저장
model_file = os.path.join(model_dir, 'deeplabv3_xception_tf_dim_ordering_tf_kernels.h5')

# PixelLib가 제공하는 모델의 url입니다.
model_url = 'https://github.com/ayoolaolafenwa/PixelLib/releases/download/1.1/deeplabv3_xception_tf_dim_ordering_tf_kernels.h5'

# 다운로드를 시작합니다
## urllib 패키지 내에 있는 request 모듈의 urlretrieve 함수를 이용해서 model_url에 있는 파일을 다운로드 해서 model_file 파일명으로 저장
urllib.request.urlretrieve(model_url, model_file)

```shell
('/aiffel/aiffel/human_segmentation/models/deeplabv3_xception_tf_dim_ordering_tf_kernels.h5',
 <http.client.HTTPMessage at 0x7f9912164580>)
```

그 다음 다운로드한 모델을 이용하여 `PixelLib`로 우리가 사용할 세그멘테이션 모델을 생성

In [ ]:
# PixelLib 라이브러리에서 가져온 클래스로 semantic segmenation을 수행하는 클래스 인스턴스를 만듦
model = semantic_segmentation()
# pascal voc에 대해 훈련된 예외모델 (model_file)을 로드하는 함수를 호출
model.load_pascalvoc_model(model_file)

모델이 이미지 입력

In [ ]:
# segmenatAsPascalvoc() 함수를 호출하여 입력된 이미지를 분할
## 분할 출력의 배열을 가져옴, 분할은 pascalvoc 데이터로 학습된 모델을 이용
segvalues, output = model.segmentAsPascalvoc(img_path)

모델마다 학습시킨 데이터셋에 따라 결과값이 달라지므로 어떤 데이터를 학습시켰는지 확인하는 것은 매우 중요하다.</br>

[PASCAL VOC](http://host.robots.ox.ac.uk/pascal/VOC/)</br>

PASCAL VOC 데이터의 라벨 종류는 아래와 같다.

In [ ]:
LABEL_NAMES = [
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
]
len(LABEL_NAMES)

```shell
21
```

목적: 사람(`person`) 라벨 추출(`15`)</br>

아까 모델에서 나온 출력값을 살펴보자.

In [ ]:
# segmentAsPascalvoc() 함수를 호출하여 입력된 이미지를 분할한 뒤 나온 결과값 중
## output을 matplotlib를 이용해 출력
plt.imshow(output)
plt.show()

![model output](../images/17.png)</br>

In [ ]:
# segmentAsPascalvoc() 함수를 호출하여 입력된 이미지를 분할한 뒤 나온 결과값 중 배열값을 출력
segvalues

```shell
{'class_ids': array([ 0,  9, 15]), 
 'masks': array([[False, False, False, ..., False, False, False], 
        [False, False, False, ..., False, False, False], 
        [False, False, False, ..., False, False, False], 
        ..., 
        [False, False, False, ..., True,  True,  True], 
        [False, False, False, ..., True,  True,  True], 
        [False, False, False, ..., True,  True,  True]])}
```

In [ ]:
# segvalues에 있는 class_ids에 담겨있는 값을 통해 pascalvoc에 담겨있는 라벨을 출력
for class_id in segvalues['class_ids']:
    print(LABEL_NAMES[class_id])

```shell
background
chair
person
```

`output`: 세그멘테이션이 된 결과가 각각 다른 색상으로 담겨있음</br>
`segvalues`: `class_ids`, `masks`</br>
    - `class_ids`를 통해 어떤 물체가 있는지 알 수 있음

이제 물체마다 `output`에 어떤 색상으로 나타나 있는지만 알아내보자

In [ ]:
# 아래 코드를 이해하지 않아도 좋습니다.
# PixelLib에서 그대로 가져온 코드입니다.
# 주목해야 할 것은 생성 코드 결과물이에요!

# 컬러맵 만들기
colormap = np.zeros((256, 3), dtype=int)
ind = np.arange(256, dtype=int)

for shift in reversed(range(8)):
    for channel in range(3):
        colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

# 생성한 20개의 컬러맵 출력
colormap[:20]

```shell
array([[  0,   0,   0], 
       [128,   0,   0], 
       [  0, 128,   0], 
       [128, 128,   0],
       [  0,   0, 128],
       [128,   0, 128],
       [  0, 128, 128],
       [128, 128, 128],
       [ 64,   0,   0],
       [192,   0,   0],
       [ 64, 128,   0],
       [192, 128,   0],
       [ 64,   0, 128],
       [192,   0, 128],
       [ 64, 128, 128],
       [192, 128, 128],
       [  0,  64,   0],
       [128,  64,   0],
       [  0, 192,   0],
       [128, 192,   0]])
```

`PixelLib`로 표현한 `15`번째 색상도 확인해보자

In [ ]:
# 컬러맵 15에 해당하는 배열 출력 (pascalvoc에 LABEL_NAMES 15번째인 사람)
colormap[15]

```shell
array([192, 128, 128])
```

주의할 것은 `output` 이미지가 **BGR** 순서로 채널 배치가 되어있다는 점</br>
`colormap`은 RGB 순서다.</br>

우리가 추출해야하는 색상 값은 아래처럼 변환이 있어야한다.

In [ ]:
# 색상순서 변경
seg_color = (128, 128, 192)

이제 `seg_color`로만 이루어진 마스크를 만들어보자

In [ ]:
# output의 픽셀별로 색상이 seg_color와 같다면 True, 다르면 False
# seg_color 값이 person의 값이므로 사람 위치를 제외하면 gray로 출력
# cmap 값을 변경하면 다른 색상으로 확인이 가능함
seg_map = np.all(output==seg_color, axis=-1)
print(seg_map.shape)
plt.imshow(seg_map, cmap=gray)
plt.show()

```shell
(500, 892)
```

![grayscale segment](../images/18.png)</br>

원래 이미지와 겹쳐보면 세그멘테이션이 얼마나 잘 되었는지 알아보기 쉬울 것 같네요.

In [ ]:
# 원본이미지를 img_show에 할당한 뒤 이미지 사람이 있는 위치와 배경을 분리해서 표현한 color_mask를 만든 뒤 두 이미지를 합쳐서 출력
img_show = img_orig.copy()

# True값을 255, False 값을 0으로 변경
img_mask = seg_map.astype(np.unit8) * 255

# 255와 0을 적당한 색상으로 바꿔보기
color_mask = cv2.applyColorMap(img_mask, cv2.COLORMAP_JET)

# 원본 이미지와 마스트를 적당히 합치기
# 0.6, 0.4는 두 이미지를 섞는 비율
img_show = cv2.addWeighted(img_show, 0.6, color_mask, 0.4, 0.0)

plt.imshow(cv2.cvtColor(img_show, cv2.COLOR_BGR2RGBß))
plt.show()

![image segmenation & addweight](../images/19.png)</br>

## 9-5. 쎨로우 포커스 만들기
### (4) 배경 흐리게 하기

이번에는 배경을 흐리게 만듭니다. `blur()` 함수를 이용합니다.

In [ ]:
# (13, 13)은 blurring kernel size를 뜻합니다.
# 다양하게 바꿔보세요
img_orig_blur = cv2.blur(img_orig, (13, 13))

plt.imshow(cv2.cvtColor(img_orig_blur, cv2.COLOR_BGR2RGB))
plt.show()

![image blur example](../images/20.png)</br>

흐려진 이미지에서 세그멘테이션 마스크를 이용해서 배경만 추출하겠습니다.

In [ ]:
# cv2.cvtColor(입력 이미지, 색상 변환 코드): 입력 이미지의 색상 채널을 변경
# cv2.COLOR_BGR2RGB: BGR 형색을 RGB형식으로 변경
img_mask_color = cv2.cvtColor(img_mask, cv2.COLOR_GRAY2BGR)

# cv2.bitwise_not(): 이미지 반전
# 배경 0, 사람 255 --> 배경 255, 사람 0
img_bg_mask = cv2.bitwise_not(img_mask_color)

# cv2.bitwise_and()로 배경만 있는 영상을 얻을 수 있다.
# 값이 0인 픽셀을 제외하는 방식으로 특정 이미지(배경)만 획득하는 방식
img_bg_blur = cv2.bitwise_and(img_orig_blur, img_bg_mask)

plt.imshow(cv2.cvtColor(img_bg_blur, cv2.COLOR_BGR2RGB))
plt.show()

![result of img_bg example](../images/21.png)</br>

## 9-6. 셸로우 포커스 만들기
### (5) 흐린 배경과 원본 영상 합성

이제 배경 영상과 사람 영상을 합치기만 하면 되겠죠?

In [ ]:
# np.where(조건, 조건이 참일때, 조건이 거짓일때)
img_concat = np.where(img_mask_color==255, img_orig, img_bg_blur)

plt.imshow(cv2.cvtColor(img_concat, cv2.COLOR_BGR2RGB))
plt.show()

![fin example](../images/22.png)</br>